# Demonstration of phenotypic traits

## Setup

This section sets up imports and methods that we need to run the simulations.

In [0]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels as sm
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
sns.set_style("white")
sns.set_context("talk", font_scale=1.5)

In [0]:
def generate_population(n, u, s, cutoff, direction):    
    loc = u
    scale = s
    parent_heights = sp.random.normal(loc=loc, scale=scale, size=n)
    cutoff = pd.Series(cutoff)
    minn = np.min(cutoff)
    maxx = np.max(cutoff)
    selected_parents = {"lt": parent_heights[parent_heights < minn], 
                        "gt": parent_heights[parent_heights > minn],
                        "ge": parent_heights[parent_heights >= minn],
                        "le": parent_heights[parent_heights <= minn],
                        "ne": parent_heights[parent_heights != minn],
                        "le_ge": parent_heights[(parent_heights <= minn) | (parent_heights >= maxx)],
                        "lt_gt": parent_heights[(parent_heights < minn) | (parent_heights > maxx)]}[direction]
    return parent_heights, selected_parents, cutoff

def plot_hist(d, label):
    plt.hist(d, alpha=0.5, label=label)
    sns.despine()
    
    
def offspring_height(x):
        d = sp.random.uniform(0, 0.2)
        if sp.random.random() < 0.5:
            return x - x*d
        else:
            return x + x*d
    
def get_offspring(h, selected_parents):
    h2 = h
    offspring_heights = []

    for p in selected_parents:
        r = sp.random.random()
        if r < h2:
            offspring_heights.append(offspring_height(p))
        else:
            offspring_heights.append(sp.random.normal(scale=scale, loc=loc))
    
    return offspring_heights

## Generate and select parents

The idea behind this experiment is to explore the relationships between $R$, $S$, and $h^2$ (narrow-sense heritability).  You will recall that:

* $R = h^2S$   
* $V_G = V_A + V_D + V_I$
* $V_P = V_G + V_E$

Here's what we'll do:

1. Create a theoretical population of $N$ individuals having some distribution of traits (a mean $\pm$ a standard deviation)
1. Define some cutoff for selection (e.g., only breed large individuals)

In [0]:
num = 20000
loc = 50
scale = 15
cutoff = [10,80]
direction = "lt_gt" # "lt", "gt", "ge", "le", "ne", "lt_gt", "le_ge"

parent_heights, selected_parents, cutoff = generate_population(num, loc, scale, cutoff, direction)

plot_hist(parent_heights, "All parents ($\mu=%.2f \pm %.3f$)" % (np.mean(parent_heights), np.std(parent_heights)))
plot_hist(selected_parents, "Selected parents ($\mu=%.2f$)" % np.mean(selected_parents))
plt.title("Selected %d out of %d for breeding" % (len(selected_parents), len(parent_heights)))
plt.legend(loc=2);

## 1. Given heritibility of our trait, simulate offspring

## 2. Determine relationship between parents (selected) and their offspring

In [0]:
offspring_heights = get_offspring(0.7, selected_parents)
reg = sp.stats.linregress(selected_parents, offspring_heights)
slope, intercept, r_value, p_value, std_err = reg
plt.scatter(selected_parents, offspring_heights)
plt.xlabel("Parent height (cm)")
plt.ylabel("Offpring height (cm)")
plt.title("y=%.5fx + %.5f ($r^2$ = %.2f, $p$ = %.5f), n=%d" % (slope, 
                                                         intercept, 
                                                         r_value**2,
                                                         p_value,
                                                         len(offspring_heights)))
plt.plot(parent_heights, parent_heights*slope+intercept, 'r-')
sns.despine()
plt.show()

# $R = h^2S$

In [0]:
S = np.mean(selected_parents) - np.mean(parent_heights)
S

In [0]:
R = slope*S
R

In [0]:
new_mean = np.mean(parent_heights) + R
new_mean

In [0]:
plot_hist(parent_heights, "All parents ($\mu=%.2f$)" % np.mean(parent_heights))
plot_hist(selected_parents, "Selected parents ($\mu=%.2f$)" % np.mean(selected_parents))
plot_hist(offspring_heights, "F1 ($\mu=%.2f$)" % np.mean(offspring_heights))
plt.axvline(np.mean(parent_heights), label="Parent mean", color="blue", alpha=0.5)
plt.axvline(np.mean(offspring_heights), label="F1 mean", color="red", alpha=0.5)
plt.legend(loc=2);